In [19]:
# Fuzzy match suggestions
import difflib

# Add helper functions
import json

FILE_PATH = "/content/drive/MyDrive/Projects/Coffee Machine/oop_version/data.json"

def load_data(filename_or_path=FILE_PATH):
  with open(filename_or_path, "r") as f:
    return json.load(f)

def save_data(data, filename_or_path=FILE_PATH):
  with open(filename_or_path, "w") as f:
    json.dump(data, f, indent=4)

# Define "MenuItem" class
class MenuItem:
  def __init__(self, name, water, milk, coffee, cost):
    self.name = name
    self.cost = cost
    self.ingredients = {
        "water": water,
        "milk": milk,
        "coffee": coffee
    }

# Define "Menu" class
class Menu:
  def __init__(self):
    self.menu = [
        MenuItem(name="latte", water=200, milk=150, coffee=24, cost=2.5),
        MenuItem(name="espresso", water=50, milk=0, coffee=18, cost=1.5),
        MenuItem(name="cappuccino", water=250, milk=50, coffee=24, cost=3)
    ]

  def get_items(self):
    return ", ".join(item.name for item in self.menu)

  def find_drink(self, order_name):
      for item in self.menu:
        if item.name == order_name:
          return item
      print("Sorry that item is not available.")

# Define "CoffeeMaker" class
class CoffeeMaker:
  def __init__(self, data):
    self.resources = data["resources"]

  def report(self):
    print(f"Water: {self.resources['water']}ml")
    print(f"Milk: {self.resources['milk']}ml")
    print(f"Coffee: {self.resources['coffee']}g")

  def is_resource_sufficient(self, drink):
    can_make = True
    for item in drink.ingredients:
      if drink.ingredients[item] > self.resources[item]:
        print(f"Sorry there is not enough {item}.")
        can_make = False
    return can_make

  def make_coffee(self, order, data):
    for item in order.ingredients:
      self.resources[item] -= order.ingredients[item]

    data["resources"] = self.resources
    save_data(data)

    print(f"Here is your {order.name} ☕️. Enjoy!")

# Define "MoneyMachine" class
class MoneyMachine:
  CURRENCY = "$"
  COIN_VALUES = {
      "quarters": 0.25,
      "dimes": 0.10,
      "nickels": 0.05,
      "pennies": 0.01
  }

  def __init__(self, data):
    self.profit = data["sales"]["profit"]
    self.money_received = 0
    self.data = data

  def report(self):
    print(f"Money: {self.CURRENCY}{self.profit}")

  def process_coins(self):
    self.money_received = 0
    print("Please insert coins.")
    for coin in self.COIN_VALUES:
      while True:
        try:
          count = int(input(f"How many {coin}?: "))
          if count < 0:
            print("Please enter a  non-negative number.")
            continue
          self.money_received += count * self.COIN_VALUES[coin]
          break
        except ValueError:
          print("Please enter a valid number.")
    return self.money_received

  def make_payment(self, cost):
    self.process_coins()
    if self.money_received >= cost:
      change = round(self.money_received - cost, 2)
      print(f"Here is {self.CURRENCY}{change} in change.")
      self.profit += cost
      self.data["sales"]["profit"] = self.profit
      self.money_received = 0
      return True
    else:
      print("Sorry that's not enough money. Money refunded.")
      self.money_received = 0
      return False

try:
  data = load_data(FILE_PATH)
except (FileNotFoundError, json.JSONDecodeError):
  print("Data file not found, initializing defaults.")
  data = {
      "resources": {
          "water": 1000,
          "milk": 1000,
          "coffee": 200
      },
      "sales": {
          "profit": 0
      }
  }
  save_data(data)

# Driver code
money_machine = MoneyMachine(data)
coffee_maker = CoffeeMaker(data)
menu = Menu()

is_on = True

while is_on:
  options = menu.get_items()
  choice = input(f"What would you like {options}: ").lower()
  if choice == 'off':
    save_data(data)
    print("Turning off... Goodbye!")
    is_on = False
  elif choice == 'report':
    coffee_maker.report()
    money_machine.report()
  elif not choice.strip():
    print("Please enter a valid option.")
    continue
  else:
    valid_options = [item.name for item in menu.menu]

    if choice not in valid_options:
      closest_match = difflib.get_close_matches(choice, valid_options, n=1, cutoff=0.6)
      if closest_match:
        confirm = input(f"'{choice}' not found. Did you mean '{closest_match[0]}'? (y/n): ").lower()
        if confirm == "y":
          choice = closest_match[0]
        else:
          print(f"'{choice}' is not available. Please choose from {options}")
          continue
      else:
        print(f"'{choice}' is not available. Please choose from {options}")
        continue

    drink = menu.find_drink(choice)
    if drink:
      if coffee_maker.is_resource_sufficient(drink) and money_machine.make_payment(drink.cost):
        coffee_maker.make_coffee(drink, data)

What would you like latte, espresso, cappucino: off
Turning off... Goodbye!
